In [2]:
import re
from collections import Counter
from pyspark import SparkContext
sc = SparkContext()

In [3]:
from pyspark.sql import SparkSession
import sys
my_spark = SparkSession \
    .builder \
    .appName("dcard") \
    .config("spark.mongodb.input.uri", "mongodb://192.168.2.12:27017/dcard.talk_posts") \
    .config("spark.mongodb.output.uri", "mongodb://192.168.2.12:27017/dcard.talk_posts") \
    .getOrCreate()

In [4]:
df = my_spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [5]:
df.printSchema()

root
 |-- _id: integer (nullable = true)
 |-- anonymousDepartment: boolean (nullable = true)
 |-- anonymousSchool: boolean (nullable = true)
 |-- commentCount: integer (nullable = true)
 |-- content: string (nullable = true)
 |-- createdAt: string (nullable = true)
 |-- department: string (nullable = true)
 |-- excerpt: string (nullable = true)
 |-- forumAlias: string (nullable = true)
 |-- forumId: string (nullable = true)
 |-- forumName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hidden: boolean (nullable = true)
 |-- hiddenByAuthor: boolean (nullable = true)
 |-- likeCount: integer (nullable = true)
 |-- media: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: string (nullable = true)
 |-- pinned: boolean (nullable = true)
 |-- replyId: integer (nullable = true)
 |-- replyTitle: string (nullable = true)
 |-- reportReason: string (nullable = true)
 |-- school: string (nullable = true)
 |-- tags: array (nullable = true

In [6]:
df.groupBy("school").count().show()

+-----------+-----+
|     school|count|
+-----------+-----+
|       新堡大學|    9|
| 國立高雄海洋科技大學|   34|
|        屏科大|  436|
|      亞當森大學|    1|
|       格魯斯特|    1|
| 國立臺南護理專科學校|   21|
|     香港浸會大學|    1|
|    love♏的♎|    1|
|         七七|    1|
|       森野小街|    1|
|   Ladycaca|    1|
|       伊比爾喬|    1|
|      理論型情聖|    1|
|        OwO|    1|
|         小隻|    1|
|💸挖壕溝的女子嘟🔨|    1|
|         米米|    1|
|     142小女紙|    2|
|        紅心K|    1|
|       新鮮な肝|    1|
+-----------+-----+
only showing top 20 rows



In [7]:
df.take(5)

[Row(_id=6150, anonymousDepartment=False, anonymousSchool=False, commentCount=19, content='好酷噢可以按讚XD', createdAt='2014-04-10T08:16:24.673Z', department='資訊工程學系', excerpt='好酷噢可以按讚XD', forumAlias='talk', forumId='255fd275-fec2-49d2-8e46-2e1557ffaeb0', forumName='閒聊', gender='M', hidden=False, hiddenByAuthor=False, likeCount=82, media=[], pinned=False, replyId=None, replyTitle='null', reportReason='', school='淡江大學', tags=[], title='新功能耶！', updatedAt='2014-04-10T08:16:24.673Z', withNickname=None),
 Row(_id=6151, anonymousDepartment=False, anonymousSchool=False, commentCount=60, content='大家快來給點建議吧 : )', createdAt='2014-04-10T09:15:11.945Z', department=' ', excerpt='大家快來給點建議吧 : )', forumAlias='talk', forumId='255fd275-fec2-49d2-8e46-2e1557ffaeb0', forumName='閒聊', gender='D', hidden=False, hiddenByAuthor=False, likeCount=18, media=[], pinned=False, replyId=None, replyTitle='null', reportReason='', school='狄卡', tags=[], title='新版建議', updatedAt='2014-04-10T09:15:11.945Z', withNickname=None),
 R

In [8]:
content = df.select('content')
content_rdd = content.rdd

In [9]:
lineLengths = content_rdd.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
print(lineLengths)
print(totalLength)

PythonRDD[25] at RDD at PythonRDD.scala:48
79631


In [10]:
content_rdd.top(2)

[Row(content='🤔🤔\n聽說今年在屏東某地的潮X高中\n全國繁星第一 (110人)\n但只有46個人上國立\n難道這就是所謂有學校就讀的概念嗎?\n\n還有據說繁星進大學的 都蠻優秀的\n是這樣嗎？'),
 Row(content='🤔剛剛要回文時已經找不到了\n總之，內容就是跟姊姊一起刮卡然後中了205萬，於是發上來的照片我怎麼看都沒中😂。\n\n祝大家新年快樂，也希望大家在刮彩卷之類的東西能夠好好看清楚喔🙌，以免遇到以為中獎一家人很high，結果被網友說沒中的哭哭戲碼。\nhttps://i.imgur.com/Svbg4BF.jpg\n沒有截文章 截到圖片而已\n\n大家加油喔👊')]

In [11]:
def remove_url(sentence):
    # remove url
    sentence = re.sub(r'^https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)

    # remove punctuation
    text_list = re.split('\W+', sentence)
    return list(filter(None, text_list))


In [12]:
test = '🤔剛剛要回文時已經找不到了\n總之，內容就是跟姊姊一起刮卡然後中了205萬，於是發上來的照片我怎麼看都沒中😂。\n\n祝大家新年快樂，也希望大家在刮彩卷之類的東西能夠好好看清楚喔🙌，以免遇到以為中獎一家人很high，結果被網友說沒中的哭哭戲碼。\nhttps://i.imgur.com/Svbg4BF.jpg\n沒有截文章 截到圖片而已\n\n大家加油喔👊'

In [13]:
remove_url(test)

['剛剛要回文時已經找不到了',
 '總之',
 '內容就是跟姊姊一起刮卡然後中了205萬',
 '於是發上來的照片我怎麼看都沒中',
 '祝大家新年快樂',
 '也希望大家在刮彩卷之類的東西能夠好好看清楚喔',
 '以免遇到以為中獎一家人很high',
 '結果被網友說沒中的哭哭戲碼',
 '沒有截文章',
 '截到圖片而已',
 '大家加油喔']

In [28]:
content_rdd.take(10)

[Row(content='好酷噢可以按讚XD'),
 Row(content='大家快來給點建議吧 : )'),
 Row(content='或用撰寫新文章回應打成長長一篇！'),
 Row(content='個人覺得喜歡的按鈕功能很惹人煩\n不需要上個網全世界都要按讚吧QAQ\n不認為喜歡鈕的新增是個好主意...'),
 Row(content='https://www.youtube.com/watch?v=JAjXKnFoaZI\n\n2014臺大畢業季主題曲「還記得嗎」\n\n詞：Crispy脆樂團\n曲：Skippy\n\n這條路我走了四年 總仰望盡頭的藍天\n兩排的椰子樹就像侍衛 守護著我們的從前\n總有一天我們要說再見 走入大人的世界\n倒影在湖面上的時間 又悄悄走了幾圈\n\n還記得嗎 對爸媽說我會拿書卷獎\n在地下室等著迎接曙光\n看見你的表情和我一樣\n我笑了吧\n\n還記得嗎 鏡子裡整齊劃一的模樣\n一個人躲在房間練吉他\n拼了命跟上大家的步伐\n我大聲唱\n"時間不能夠重來 我不要留下一分一秒的空白"\n\n你對我說面對新的生活 有些不安和惶恐\n是會一飛沖天變成英雄 還是流星般墜落\n別害怕 相信自己 和那些倔強執著\n每個人都是一片天空 綻放自己的煙火\n\n還記得嗎 一起翹課占領籃球場\n在宿舍電腦裡釋放魔法\n在哪裡跌倒哪裡受了傷\n我不會怕\n\n還記得嗎 一起走在圖書館的長廊\n在耳邊小小聲說著夢想\n空無一人的椰林大道上\n我大聲唱\n"想要改變全世界 這裡就是我們的 起點"\n\n還記得嗎 第一次遇見她的地方\n牽著手散步在舟山路上\n不再去在意別人的眼光\n我吻了她\n\n還記得嗎 對鏡子告訴自己要堅強\n不管未來會在哪個地方\n在人擠人的畢業舞會上\n換上西裝\n"人生不需要100分 能和你們一起走過就值得"'),
 Row(content='(轉stupidclown)\n我的朋友是外地人，第一次來台北坐捷運，怕搭錯方向，就問了一個在月台上的媽媽。\n他問：請問是往永寧嗎？\n媽媽回：不好意思你認錯人了。\n\n\n難道不是都會這樣嗎QQ'),
 Row(content='考前體悟到系上分三種人\n\n上完課就懂\n\n考前懵懵懂懂\n\n從來都不懂\n\n第三種人會隨年級數減少\n\n\n\n\n\n\n\n\n\n

In [30]:
result = content_rdd.map(lambda s: remove_url(s['content'])).collect()


In [31]:
result[:10]

[['好酷噢可以按讚XD'],
 ['大家快來給點建議吧'],
 ['或用撰寫新文章回應打成長長一篇'],
 ['個人覺得喜歡的按鈕功能很惹人煩', '不需要上個網全世界都要按讚吧QAQ', '不認為喜歡鈕的新增是個好主意'],
 ['2014臺大畢業季主題曲',
  '還記得嗎',
  '詞',
  'Crispy脆樂團',
  '曲',
  'Skippy',
  '這條路我走了四年',
  '總仰望盡頭的藍天',
  '兩排的椰子樹就像侍衛',
  '守護著我們的從前',
  '總有一天我們要說再見',
  '走入大人的世界',
  '倒影在湖面上的時間',
  '又悄悄走了幾圈',
  '還記得嗎',
  '對爸媽說我會拿書卷獎',
  '在地下室等著迎接曙光',
  '看見你的表情和我一樣',
  '我笑了吧',
  '還記得嗎',
  '鏡子裡整齊劃一的模樣',
  '一個人躲在房間練吉他',
  '拼了命跟上大家的步伐',
  '我大聲唱',
  '時間不能夠重來',
  '我不要留下一分一秒的空白',
  '你對我說面對新的生活',
  '有些不安和惶恐',
  '是會一飛沖天變成英雄',
  '還是流星般墜落',
  '別害怕',
  '相信自己',
  '和那些倔強執著',
  '每個人都是一片天空',
  '綻放自己的煙火',
  '還記得嗎',
  '一起翹課占領籃球場',
  '在宿舍電腦裡釋放魔法',
  '在哪裡跌倒哪裡受了傷',
  '我不會怕',
  '還記得嗎',
  '一起走在圖書館的長廊',
  '在耳邊小小聲說著夢想',
  '空無一人的椰林大道上',
  '我大聲唱',
  '想要改變全世界',
  '這裡就是我們的',
  '起點',
  '還記得嗎',
  '第一次遇見她的地方',
  '牽著手散步在舟山路上',
  '不再去在意別人的眼光',
  '我吻了她',
  '還記得嗎',
  '對鏡子告訴自己要堅強',
  '不管未來會在哪個地方',
  '在人擠人的畢業舞會上',
  '換上西裝',
  '人生不需要100分',
  '能和你們一起走過就值得'],
 ['轉stupidclown',
  '我的朋友是外地人',
  '第一次來台北坐捷運',
  '怕搭錯方向',
  '就問了一個在月台上的媽媽',
  '

In [37]:
def to_ngrams(unigrams, length):
    return zip(*[unigrams[i:] for i in range(length)])

In [46]:
# compute unigram and bigram count
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for post in result:
    for line in post:
        unigram_counter.update(line)
        bigram_counter.update(to_ngrams(line, 2))
        trigram_counter.update(to_ngrams(line, 3))

In [48]:
print(unigram_counter.most_common(10))
print(bigram_counter.most_common(10))
print(trigram_counter.most_common(10))


[('的', 543687), ('我', 347812), ('是', 329005), ('不', 252695), ('一', 239275), ('有', 237782), ('了', 172307), ('人', 143985), ('在', 137319), ('到', 124767)]
[(('沒', '有'), 43618), (('什', '麼'), 41772), (('可', '以'), 39754), (('一', '個'), 37991), (('自', '己'), 37321), (('知', '道'), 36272), (('大', '家'), 35983), (('覺', '得'), 34301), (('因', '為'), 33830), (('真', '的'), 32435)]
[(('不', '知', '道'), 18466), (('的', '時', '候'), 15026), (('有', '沒', '有'), 13611), (('_', '_', '_'), 13133), (('為', '什', '麼'), 9418), (('自', '己', '的'), 8194), (('哈', '哈', '哈'), 7108), (('我', '覺', '得'), 6325), (('是', '不', '是'), 6131), (('真', '的', '很'), 6026)]
